In [3]:
import json

In [4]:
all_data = json.loads(open('mega_data.json').read())

In [6]:
# let's get these keyed by hwId and questionNumber

data_by_problem = {}

for d in all_data:
    # filter out non-cli responses
    if d['origin'] != 'cli':
        continue
    
    hwId = d['hwId']
    questionNumber = d['questionNumber']
    email = d['email']
    if hwId not in data_by_problem:
        data_by_problem[hwId] = {}
    if questionNumber not in data_by_problem[hwId]:
        data_by_problem[hwId][questionNumber] = {}
    if email not in data_by_problem[hwId][questionNumber]:
        data_by_problem[hwId][questionNumber][email] = []
    data_by_problem[hwId][questionNumber][email].append(d)

In [7]:
data_by_problem[3][3]['8b44999349148e2e02a6aca6ab24365a9e7f2795bb1f59643b0dc7bb7f7fcd43']

[{'requestId': 'HW03 Q3 (8b44999349148e2e02a6aca6ab24365a9e7f2795bb1f59643b0dc7bb7f7fcd43--RC) @ Fri Feb 16 2024 15:46:56 GMT-0800 (Pacific Standard Time) from interleaved_sum',
  'feedback': {'timestamp': '2024-02-16T23:54:31.650Z', 'feedback': '0'},
  'assessment': None,
  'hwId': 3,
  'questionNumber': 3,
  'email': '8b44999349148e2e02a6aca6ab24365a9e7f2795bb1f59643b0dc7bb7f7fcd43',
  'consent': True,
  'timestamp': 1708127216000,
  'activeFunction': 'interleaved_sum',
  'prompt': 'You are a very talented 61a tutoring bot, the intro CS class at UC Berkeley; you are helping students learn to program. \n\nA student has asked for help. The question they are trying to solve is in the next message, and in the following message is (all) the code the student has so far, including some code that is for other questions -- ignore that other code. If the student asks for help repeatedly, the conversation will continue with your subsequent reply and any updated student code.\n\nFirst: identify 

In [36]:
# sample 20 sequnces by picking a random hw id, a random question number, and a random email

import random

random.seed(72)

sample = []

INCLUDES_FEEDBACK = '3'
SEQUENCE_MINIMUM = 3

for i in range(20):
    retry = True
    while retry:
        hwId = random.choice([1,2,3,4,5]) # only python questions for now
        questionNumber = random.choice([q for q in data_by_problem[hwId].keys() if q != 'N/A'])
        email = random.choice(list(data_by_problem[hwId][questionNumber].keys()))
        sequence = data_by_problem[hwId][questionNumber][email]
        if len(sequence) > SEQUENCE_MINIMUM and \
                any([d['feedback']['feedback'][:len(INCLUDES_FEEDBACK)] == INCLUDES_FEEDBACK 
                     for d in sequence if 'feedback' in d and d['feedback'] is not None]):
            sample.append(data_by_problem[hwId][questionNumber][email])
            retry = False

def extract_single_function(code, function):
    # only python questions for now
    
    code = code.split('\n')
    in_function = False
    function_depth = 0
    function_code = []
    for line in code:
        if in_function:
            if line.strip().startswith('def ') and line.index('d') <= function_depth:
                break
            function_code.append(line)
        elif line.strip().startswith('def ' + function):
            function_code.append(line)
            in_function = True
            function_depth = line.index('d')
    return '\n'.join(function_code)

for s in sample:
    # print the sequence by looking at each of the code/answer pairs
    print ('===\n===\nHW', s[0]['hwId'], 'Question', s[0]['questionNumber'], '('+s[0]['activeFunction']+')', 'Email', s[0]['email'])
    starttime = s[0]['timestamp']
    for i, d in enumerate(s):
        reltime = (d['timestamp'] - starttime) / 60000
        feedback = d['feedback']
        if feedback is not None:
            feedback = feedback['feedback']
        print('===== ' + str(i) + ' ===== @ ' + f'{reltime:.2f}m' + ' ('+d['activeFunction']+', Q'+str(d['questionNumber'])+') feedback: ' + str(feedback))
        print(extract_single_function(d['code'], d['activeFunction']))
        print('===')
        print(d['output'])
        print()

===
===
HW 2 Question 2 (accumulate) Email 422f4946b1e5286e5798a60502b90f932e9a7b7c7affefbb76714c2a19a42962
===== 0 ===== @ 0.00m (accumulate, Q2) feedback: 1
def accumulate(fuse, start, n, term):
    """Return the result of fusing together the first n terms in a sequence 
    and start.  The terms to be fused are term(1), term(2), ..., term(n). 
    The function fuse is a two-argument commutative & associative function.

    >>> accumulate(add, 0, 5, identity)  # 0 + 1 + 2 + 3 + 4 + 5
    15
    >>> accumulate(add, 11, 5, identity) # 11 + 1 + 2 + 3 + 4 + 5
    26
    >>> accumulate(add, 11, 0, identity) # 11 (fuse is never used)
    11
    >>> accumulate(add, 11, 3, square)   # 11 + 1^2 + 2^2 + 3^2
    25
    >>> accumulate(mul, 2, 3, square)    # 2 * 1^2 * 2^2 * 3^2
    72
    >>> # 2 + (1^2 + 1) + (2^2 + 1) + (3^2 + 1)
    >>> accumulate(lambda x, y: x + y + 1, 2, 3, square)
    19
    """
    "*** YOUR CODE HERE ***"
    x = 1
    while x < n + 1:
        o = fuse(start,term(x))
  